# ensemble_emotion_simulation

In [32]:
import glob
import librosa
import librosa.display
import numpy as np
import _pickle as pickle
import pandas as pd

In [33]:
classes = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs, chroma, mel, contrast, tonnetz


target_files = []

In [34]:
def parse_audio_files(path):
    features = np.empty((0, 193))
    for fn in glob.glob(path):
        try:
            mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
        except Exception as e:
            print("Error encountered while parsing file: ", fn)
            continue
        ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        features = np.vstack([features, ext_features])
        target_files.append(fn)
    return np.array(features)

In [36]:
ts_features = parse_audio_files('C:/Users/Dell/Documents/Project Files/new_test_sounds/*.wav')
tr_features = np.array(ts_features, dtype=pd.Series)

filename = 'C:/Users/Dell/Documents/Project Files/Ensemble_Model_protocol2.sav'
model = pickle.load(open(filename, 'rb'))

prediction = model.predict(ts_features)

for i, val in enumerate(prediction):
    print("Input File: ", target_files[i], "|", " Predicted Emotion Is:", classes[int(val)])

Input File:  C:/Users/Dell/Documents/Project Files/new_test_sounds\YAF_luck_happy.wav |  Predicted Emotion Is: happy
Input File:  C:/Users/Dell/Documents/Project Files/new_test_sounds\YAF_rain_ps.wav |  Predicted Emotion Is: surprise
Input File:  C:/Users/Dell/Documents/Project Files/new_test_sounds\YAF_tell_angry.wav |  Predicted Emotion Is: angry


# ensemble_emotion_training

In [24]:
import glob
import librosa
import librosa.display
import numpy as np
import _pickle as pickle
from sklearn import svm
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.grid_search import GridSearchCV

In [25]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs, chroma, mel, contrast, tonnetz

In [26]:
def parse_audio_files(path):
    features, labels = np.empty((0, 193)), np.empty(0)
    labels = []
    for fn in glob.glob(path):
        try:
            mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
        except Exception as e:
            print("Error encountered while parsing file: ", fn)
            continue
        ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        features = np.vstack([features, ext_features])
        labels = np.append(labels, fn.split('/')[4].split('-')[1].split('.')[0])
    return np.array(features), np.array(labels, dtype=np.int)

In [27]:
tr_features, tr_labels = parse_audio_files('C:/Users/Dell\Documents/Project Files/train_sounds/*.wav')

tr_features = np.array(tr_features, dtype=pd.Series)
tr_labels = np.array(tr_labels, dtype=pd.Series)

In [28]:
model1 = svm.SVC(kernel='linear', C=1000, gamma='auto')
model1.fit(X=tr_features.astype(int), y=tr_labels.astype(int))

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
model2 = svm.SVC(kernel='rbf', C=1000, gamma='auto')
model2.fit(X=tr_features.astype(int), y=tr_labels.astype(int))

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [30]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
estimators = [('linear', model1), ('rbf', model2)]
ensemble = VotingClassifier(estimators)

In [31]:
results = model_selection.cross_val_score(ensemble, tr_features.astype(int), tr_labels.astype(int), cv=kfold)
ensemble.fit(X=tr_features.astype(int), y=tr_labels.astype(int))

filename = ('C:/Users/Dell/Documents/Project Files/Ensemble_Model_protocol2.sav')

pickle.dump(ensemble, open(filename, 'wb'), protocol=2)

print('Your Model has Saved..')
print ('cheers   :)')

Your Model has Saved..
cheers   :)
